In [ ]:
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy
import requests
import time

In [ ]:
cd drive/My Drive

/content/drive/My Drive


In [ ]:
class AttoCosmeScrape():

  def __init__(self, site_url):
    self.url = site_url
    self.drop_tag = re.compile(r"<[^>]*?>")

  def get_home_url_list(self):
    home_url_list = []
    for i in range(0, 3):
      home_page_url = self.url + '/page/' + str(i)
      home_url_list.append(home_page_url)
    return home_url_list

  # home_urlのテキスト取得
  def get_soup(self):
    soup_list = []
    home_url_list = self.get_home_url_list()
    for home_url in home_url_list:
      time.sleep(1)
      res = requests.get(home_url)
      soup = BeautifulSoup(res.text, 'html.parser')
      soup_list.append(soup)
    return soup_list

  #ランキング更新日取得
  def get_uproad_date(self):
    uproad_list = []
    soup_list = self.get_soup()
    for soup in soup_list:
      for div in soup.find_all('div', class_='clearfix',id="keyword-ranking-header"):
        for span in div('span'):
          rank_uproad_date = self.drop_tag.sub("",str(span))
          rank_uproad_date = rank_uproad_date.replace('更新日：','')
    return rank_uproad_date

  #各商品url取得
  def get_product_url(self):
    soup_list = self.get_soup()
    product_url_list = []
    for soup in soup_list:
      product_lists = soup.find_all('dd',class_="pic")
      for product_list in product_lists:
        for a in product_list.select('a'):
          product_url = a.attrs['href']
          product_url_list.append(product_url)
    return product_url_list

  #各商品の各レビューurl取得
  def get_product_all_review_url(self):
    product_url_list = self.get_product_url()
    product_all_review_url_list = []
    for pro_url in product_url_list:
      time.sleep(1)
      res = requests.get(pro_url)
      soup = BeautifulSoup(res.text, 'html.parser')
      li = soup.find('li', class_="review")
      a = li.find('a')
      product_all_review_url = a.attrs['href']
      for i in range(0, 3):
        pro_all_review_page_url = product_all_review_url + '/p/' + str(i)
        product_all_review_url_list.append(pro_all_review_page_url)
    return product_all_review_url_list

  #各商品の各レビュー取得
  def get_product_review(self, product_all_review_url):
    text_list = []
    product_all_review_url_list = product_all_review_url
    for pro_all_review in product_all_review_url_list:
      time.sleep(1)
      res = requests.get(pro_all_review)
      soup = BeautifulSoup(res.text, 'html.parser')
      for span in soup.find_all('span', class_="read-more"):
        for a in span.find_all('a'):
          review_more_url = a.attrs['href']
          time.sleep(1)
          res = requests.get(review_more_url)
          soup = BeautifulSoup(res.text, 'html.parser')
          text = soup.find_all('p', class_="read")
          text = self.drop_tag.sub("", str(text))
          text = text.strip()
          text_list.append(text)
    return text_list

  #レビューを行った人のデモグラ変数取得
  def get_demogura(self, product_all_review_url):
    age_list = []
    face_type_list = []
    product_name_list = []
    product_score_list = []
    reviewer_score_list = []

    product_all_review_url_list = product_all_review_url

    for pro_all_review in product_all_review_url_list:
      time.sleep(1)
      res = requests.get(pro_all_review)
      soup = BeautifulSoup(res.text, 'html.parser')

      for span in soup.find_all('span', class_="read-more"):
        for a in span.find_all('a'):
          review_more_url = a.attrs['href']
          time.sleep(1)
          res = requests.get(review_more_url)
          soup = BeautifulSoup(res.text, 'html.parser')

          #レビューした商品名
          product_name = soup.find_all('strong', class_='pdct-name fn')
          product_name = self.drop_tag.sub("",str(product_name))
          product_name = product_name.replace('[', '')
          product_name = product_name.replace(']', '')
          product_name_list.append(product_name)

          #レビューした商品のスコア
          pd_score_div = soup.find('div', class_='info-desc')
          product_score = pd_score_div.find('p')
          product_score = self.drop_tag.sub("", str(product_score))
          product_score = product_score.replace('[', '')
          product_score = product_score.replace(']', '')
          product_score_list.append(product_score)

          #レビューした人の商品に対するスコア
          reviewer_score_div = soup.find('div', class_='rating clearfix')
          reviewer_score = self.drop_tag.sub("", str(reviewer_score_div))
          reviewer_score = re.search(r'\d', reviewer_score)
          reviewer_score = reviewer_score.group()
          reviewer_score_list.append(reviewer_score)

          for div in soup.find_all('div', class_="reviewer-info"):
            demogura = div.find_all('li')
            demogura = self.drop_tag.sub("",str(demogura))

            #レビューした人の年齢取得
            if '歳' in demogura:
              age = re.search(r'\d.*歳', demogura)
              age = age.group()
            else:
              age = 'Null'
            age_list.append(age)

            #レビューした人の肌タイプを取得
            if '肌' in demogura:
              face_type = re.search(r'..肌', demogura)
              face_type = face_type.group()
            else:
              face_type = 'Null'
            face_type_list.append(face_type)

    return product_name_list, product_score_list, reviewer_score_list, age_list, face_type_list  
   

In [ ]:
home_url = 'https://www.cosme.net/item/item_id/902/ranking'
scrape = AttoCosmeScrape(home_url)
product_all_review_url_list = scrape.get_product_all_review_url()


In [ ]:
input_date = scrape.get_uproad_date()
text_list = scrape.get_product_review(product_all_review_url_list)

In [ ]:
text_list

['[\nインナードライと診断され、肌の基盤を作りたいと思い購入してみました。パシャパシャとした使い心地で使いやすかったです。効果は、よくわからずわたしには保湿が足りなかったようなので他のものに切り替えてみようと思います。\n\n]',
 '[\n化粧水を塗った段階でかなり保湿力を感じました。荒れて敏感なところに塗っても平気でした。\n\n]',
 '[\n3本目のリピートです。乾燥がひどい時でもしみる事なくすっと肌に馴染みます。コットンパックするとより浸透する感じ。\n\n]',
 '[\nランキング上位でずっと気になっていて、手持ちの化粧水が切れそうなタイミングで購入しました。これはランキング上位なのも分かります！シャバシャバ系なのに付けた後はしっとりで、乾燥もしにくいです。無香料好きなのでわたしに合ってるんですね。マスクで出来てしまったニキビが落ち着いた&amp;出来にくくなってます。生理時期に肌が荒れるのですが、その時でも良かったらリピします！\n\n]',
 '[\n口コミを見て気になり購入しました。サラサラしているのにしっかり浸透してちゃんと保湿されます！にきびにもいいという口コミもありましたが、私の場合はにきびが良くなるとかはなかったです、、少し減りが早いように感じるのでコスパはあまり良くないかなと思いますが、使用感はとても良いです！\n\n]',
 '[\n保湿効果があるのにベタベタしなくて使いやすい！ニキビも出来にくい気がします。ずっと使い続けたいと思います！\n\n]',
 '[\nさすが！人気だけあって良かったです！これだけでしっとり保湿され乾燥を感じませんでした。あえて改善してほしいところを言うなら..これはこれで見た目は良いのですが、ドバッと勢いよく出て溢すことがあるのでポンプ式なら良いのになと思いました。\n\n]',
 '[\n元々ニキビが出来やすい肌でもあり敏感肌でもある肌なので最後の砦だと思って使うことに決めたIPSAさんのザタイムRアクア。使用から3日目で顎のざらざらやベタつき？がなくなっていて驚きました…！！！肌全体も薄い膜を張っているかのような肌触りに。ニキビ跡や肌の赤みも和らいだような感じがします。お値段は少し張りますが使い続けたいなと思う化粧水です！\n\n]',
 '[\n好きなアイドルの方が使用してたので買いました。ボ

In [ ]:
product_name_list, product_score_list, reviewer_score_list, age_list, face_type_list = scrape.get_demogura(product_all_review_url_list)

In [ ]:
print('レビュー数: ', len(text_list))
print('商品数: ', len(product_name_list))
print('年齢変数数: ', len(age_list))
print('レビュースコア数: ', len(reviewer_score_list))

レビュー数:  900
商品数:  900
年齢変数数:  900
レビュースコア数:  900


## DataFrame作成

In [ ]:
for i in range(len(text_list)):
  text_list[i] = text_list[i].replace('\n', '')
  text_list[i] = text_list[i].replace('[', '')
  text_list[i] = text_list[i].replace(']', '')

In [ ]:
df = pd.DataFrame(product_name_list, columns={'product_name'})

In [ ]:
df['product_score'] = product_score_list
df['age'] = age_list
df['face_type'] = face_type_list
df['reviewer_score'] = reviewer_score_list
df['review'] = text_list
df['input_date'] = input_date

### 変数の型編集

In [ ]:
def change_valiable_type(dataframe):
  
  dataframe['product_score'] = dataframe['product_score'].astype(float)
  dataframe['reviewer_score'] = dataframe['reviewer_score'].astype(float)
  dataframe['age'] = dataframe['age'].str.strip('歳')
  dataframe['age'] = dataframe['age'].astype(int)
  dataframe['input_date'] = pd.to_datetime(dataframe['input_date'])
  return dataframe

In [ ]:
df = change_valiable_type(df)

In [ ]:
df

,product_name,product_score,age,face_type,reviewer_score,review,input_date
0,ザ・タイムR アクア,5.1,27,混合肌,3.0,インナードライと診断され、肌の基盤を作りたいと思い購入してみました。パシャパシャとした使い心...,2020-11-20
1,ザ・タイムR アクア,5.1,38,乾燥肌,5.0,化粧水を塗った段階でかなり保湿力を感じました。荒れて敏感なところに塗っても平気でした。,2020-11-20
2,ザ・タイムR アクア,5.1,41,混合肌,4.0,3本目のリピートです。乾燥がひどい時でもしみる事なくすっと肌に馴染みます。コットンパックする...,2020-11-20
3,ザ・タイムR アクア,5.1,34,敏感肌,6.0,ランキング上位でずっと気になっていて、手持ちの化粧水が切れそうなタイミングで購入しました。こ...,2020-11-20
4,ザ・タイムR アクア,5.1,21,混合肌,4.0,口コミを見て気になり購入しました。サラサラしているのにしっかり浸透してちゃんと保湿されます！...,2020-11-20
...,...,...,...,...,...,...,...
895,プロP ミスト,5.2,37,混合肌,4.0,粒子が細かく気持ち良い。手軽で使いやすいです。これだけで潤う…とまではいかないので、ブースタ...,2020-11-20
896,プロP ミスト,5.2,23,乾燥肌,6.0,頂き物ですが、使い心地が良く毎日使っているのでリピートしようかと思っています。お気に入りポイ...,2020-11-20
897,プロP ミスト,5.2,30,混合肌,5.0,かなり使い勝手がいいです。お風呂あがりにケアする前に乾燥しないようにシューっとしたり、メイク...,2020-11-20
898,プロP ミスト,5.2,28,混合肌,5.0,ミスト状の美容液で、粒子が細かく肌にすぐなじむので乾燥も和らぎます(^^)メイクの上からでも...,2020-11-20


In [ ]:
df.to_csv('@cosme_1120.csv', index=False)